# Think Bayes

This notebook presents example code and exercise solutions for Think Bayes.

Copyright 2018 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

In [18]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import classes from thinkbayes2
from thinkbayes2 import Hist, Pmf, Suite, MakeMixture

In [38]:
def add(pmf1, pmf2):
    res = Pmf()
    for v1, p1 in pmf1.Items():
        for v2, p2 in pmf2.Items():
            res[v1, v2] = p1 * p2
    return res

In [7]:
from sympy import symbols
p_citizen, p_cv, p_ncv, p_error = symbols('p_citizen, p_cv, p_ncv, p_error')

In [8]:
def make_binary(p, name1, name2):
    return Pmf({name1: p, name2: 1-p})

In [67]:
citizen_status = ['citizen', 'non-citizen']
pmf_citizen = make_binary(p_citizen, *citizen_status)

Pmf({'citizen': p_citizen, 'non-citizen': -p_citizen + 1})

In [68]:
error_status = ['error', 'no-error']
pmf_error = make_binary(p_error, *error_status)

Pmf({'error': p_error, 'no-error': -p_error + 1})

In [69]:
pmf_citizen_report = add(pmf_citizen, pmf_error)
pmf_citizen_report.Print()

('citizen', 'error') p_citizen*p_error
('citizen', 'no-error') p_citizen*(-p_error + 1)
('non-citizen', 'error') p_error*(-p_citizen + 1)
('non-citizen', 'no-error') (-p_citizen + 1)*(-p_error + 1)


In [70]:
vote_status = ['vote', 'no-vote']
pmf_cv = make_binary(p_cv, *vote_status)

Pmf({'vote': p_cv, 'no-vote': -p_cv + 1})

In [71]:
pmf_cv_report = add(pmf_cv, pmf_error)
pmf_cv_report.Print()

('no-vote', 'error') p_error*(-p_cv + 1)
('no-vote', 'no-error') (-p_cv + 1)*(-p_error + 1)
('vote', 'error') p_cv*p_error
('vote', 'no-error') p_cv*(-p_error + 1)


In [72]:
pmf_ncv = make_binary(p_ncv, *vote_status)

Pmf({'vote': p_ncv, 'no-vote': -p_ncv + 1})

In [73]:
pmf_ncv_report = add(pmf_ncv, pmf_error)
pmf_ncv_report.Print()

('no-vote', 'error') p_error*(-p_ncv + 1)
('no-vote', 'no-error') (-p_error + 1)*(-p_ncv + 1)
('vote', 'error') p_error*p_ncv
('vote', 'no-error') p_ncv*(-p_error + 1)


In [66]:
mix = Pmf()

for val1, p1 in pmf_citizen_report.Items():
    c, e = val1
    pmf = pmf_cv_report if c=='citizen' else pmf_ncv_report
    for val2, p2 in pmf.Items():
        mix[val1, val2] = p1 * p2
        
mix.Print()

(('citizen', 'error'), ('no-vote', 'error')) p_citizen*p_error**2*(-p_cv + 1)
(('citizen', 'error'), ('no-vote', 'no-error')) p_citizen*p_error*(-p_cv + 1)*(-p_error + 1)
(('citizen', 'error'), ('vote', 'error')) p_citizen*p_cv*p_error**2
(('citizen', 'error'), ('vote', 'no-error')) p_citizen*p_cv*p_error*(-p_error + 1)
(('citizen', 'no-error'), ('no-vote', 'error')) p_citizen*p_error*(-p_cv + 1)*(-p_error + 1)
(('citizen', 'no-error'), ('no-vote', 'no-error')) p_citizen*(-p_cv + 1)*(-p_error + 1)**2
(('citizen', 'no-error'), ('vote', 'error')) p_citizen*p_cv*p_error*(-p_error + 1)
(('citizen', 'no-error'), ('vote', 'no-error')) p_citizen*p_cv*(-p_error + 1)**2
(('non-citizen', 'error'), ('no-vote', 'error')) p_error**2*(-p_citizen + 1)*(-p_ncv + 1)
(('non-citizen', 'error'), ('no-vote', 'no-error')) p_error*(-p_citizen + 1)*(-p_error + 1)*(-p_ncv + 1)
(('non-citizen', 'error'), ('vote', 'error')) p_error**2*p_ncv*(-p_citizen + 1)
(('non-citizen', 'error'), ('vote', 'no-error')) p_erro

In [74]:
def report(state, alternatives):
    val, error = state
    if error != 'error':
        return val
    alt1, alt2 = alternatives
    return alt1 if val==alt2 else alt2

In [75]:
report(('citizen', 'error'), citizen_status)

'non-citizen'

In [76]:
report(('citizen', 'no-error'), citizen_status)

'citizen'

In [81]:
pmf_report = Pmf()

for (cstate, vstate), p in mix.Items():
    creport = report(cstate, citizen_status)
    vreport = report(vstate, vote_status)
    pmf_report[creport, vreport] += p

In [82]:
pmf_report.Print()

('citizen', 'no-vote') p_citizen*p_cv*p_error*(-p_error + 1) + p_citizen*(-p_cv + 1)*(-p_error + 1)**2 + p_error**2*p_ncv*(-p_citizen + 1) + p_error*(-p_citizen + 1)*(-p_error + 1)*(-p_ncv + 1)
('citizen', 'vote') p_citizen*p_cv*(-p_error + 1)**2 + p_citizen*p_error*(-p_cv + 1)*(-p_error + 1) + p_error**2*(-p_citizen + 1)*(-p_ncv + 1) + p_error*p_ncv*(-p_citizen + 1)*(-p_error + 1)
('non-citizen', 'no-vote') p_citizen*p_cv*p_error**2 + p_citizen*p_error*(-p_cv + 1)*(-p_error + 1) + p_error*p_ncv*(-p_citizen + 1)*(-p_error + 1) + (-p_citizen + 1)*(-p_error + 1)**2*(-p_ncv + 1)
('non-citizen', 'vote') p_citizen*p_cv*p_error*(-p_error + 1) + p_citizen*p_error**2*(-p_cv + 1) + p_error*(-p_citizen + 1)*(-p_error + 1)*(-p_ncv + 1) + p_ncv*(-p_citizen + 1)*(-p_error + 1)**2
